Rename a bunch of files using Git and renamer?



In [ ]:
var renamer = require('renamer');

function renameUsingGit(gitRoot, match, find, replace) {
    var files = renamer.expand(path.join(gitRoot, match));
    var results = renamer.replace({
        files: files.filesAndDirs,
        find: find,
        replace: replace
    });
    return renamer.dryRun(results).list.map(r => {
        // rename with git instead
        var output = execSync('git '
            + '--work-tree=' + JSON.stringify(gitRoot)
            + ' --git-dir=' + JSON.stringify(path.join(gitRoot, '.git'))
            + ' mv ' + JSON.stringify(r.before)
            + ' ' + JSON.stringify(r.after));
        return output.toString() || r.after;
    });
};
renameUsingGit;


In [ ]:
// test it
// rename all scss files to less
//renameUsingGit(
//    '/Users/briancullinan/Documents/studysauce4',
//    'src/**/*.css',
//    '.css',
//    '.scss');


json git tree?


In [ ]:
var execSync = require('child_process').execSync;

function gitTree(project) {
    const dir = process.cwd();
    process.chdir(project); 
    var branches = execSync('git branch --list --all ').toString()
        .trim().split(/\s*\n\s*\**\s*/ig)
        .map(b => b.replace('remotes/', '').trim());

    // reverse git so we can add merges to the "graph" after the branch tree is constructed
    var branch = execSync('git log --pretty=format:"%ct %d %h %p" '
        + '--reverse --branches --tags ').toString().split(/\s*\n\s*/ig);
    var all = []; //execSync('git log --pretty=format:"%h %p" --reverse --all').toString().split(/\s*\n\s*/ig);

    var nodes = [];
    var edges = [];
    branch.forEach(b => {
        var commits = ((/[0-9]+\s+\(.*?\)\s*(.+)/ig).exec(b)
            || (/[0-9]+\s+(.+)/ig).exec(b) || [b, b.trim()])[1].trim().split(/\s+/ig);
        commits.forEach(r => {
            if (nodes.filter(d => d.name == r).length == 0) {
                nodes[nodes.length] = {
                    name: r,
                    index: nodes.length,
                    branch: ((/[0-9]+\s+\((.*?)\)/ig).exec(b) || [b, ''])[1],
                    time: (/([0-9]+)\s+/ig).exec(b)[1]
                }
            }
        });
        // don't include merges in tree
        if (commits.length == 3) {
            commits = [commits[0], commits[2]];
        }
        if (commits.length == 2) {
            edges = edges.concat(commits.slice(1)
                // check all existing sources to test for merge
                .map((c, i) => {
                    return {source: c, target: commits[i]}; // -1 because of slice(1)
                }))
        }
    });

    process.chdir(dir);
    return formatNodes(nodes, edges.reverse());
}
module.exports = gitTree;



tip git tree?


In [10]:
var importer = require('../Core');
var _ = require('underscore');
var fs = require('fs');
var path = require('path');
var execSync = require('child_process').execSync;
var gulp = require('gulp');
var copy = require('gulp-copy');
var clean = require('gulp-clean');
var {
    formatNodes, gitTree, displayBranches,
    tasksToPromise, listInProject, formatNodes
} = importer.import([
    'd3.ipynb[format d3 tree]',
    'gulp tasks',
    'json git tree',
    'd3.ipynb[display d3 tree]',
    'list project files'
], {gulp});

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project_orig = PROFILE_PATH + '/Documents/portal';
var project = PROFILE_PATH + '/Documents/portal3';

gulp.task('clear project', function () {
    return gulp.src(project, {read: false})
        .pipe(clean({force: true}));
});

gulp.task('copy index', function () {
    return gulp.src(project_orig)
        .pipe(copy(project))
        .dest(project);
});

function getResetCommit(branch, parent) {
    const b = branch.replace('HEAD -> ', '').trim();
    execSync('git checkout ' + b);
    execSync('git reset --mixed ' + parent.replace('HEAD -> ', '').trim());
    execSync('git add -A')
    execSync('git commit -m "auto reset ' + b + '"')
}

var svgs = [];
function resetAllBranches() {
    var children = [];
    var nodeTree = _.sortBy(gitTree(project), a => a.time);
    var promises = [];
    for (const n of nodeTree) {
        var current = n;
        var parentCount = 0;
        while (typeof current.parent !== 'undefined'
        && current.parent.branch === current.branch) {
            current = current.parent;
            parentCount++;
        }
        if (parentCount > 0 && typeof current.parent !== 'undefined') {
            // TODO: keep track of the parent so the squashed branch can be merged on to the newly squashed parent
            promises.push(resolve => {
                try {
                    if (typeof n.children !== 'undefined') {
                        children.push(n);
                    }
                    getResetCommit(n.branch, current.parent.name);
                } finally {
                    var nodeTree = _.sortBy(gitTree(project), a => a.time);
                    svgs.push(displayBranches(nodeTree));
                    resolve();
                }
            });
        }
    }
    return importer.runAllPromises(promises)
        .then(() => children);
}

function gitTipOfTree() {
    svgs = [];
    const oldDir = process.cwd();
    return (fs.existsSync(project) ? tasksToPromise(['clear project']) : Promise.resolve([]))
        // do a deep copy of all git files
        .then(() => Promise.all([
            listInProject(project_orig, '**/*'),
            listInProject(project_orig + '/.git', '**/*')
        ]))
        .then(r => {
            for (const file of [].concat(...r).sort()) {
                const dirname = path.dirname(
                    file.replace(project_orig, project)).split('/');

                for (var i = 2; i <= dirname.length; i++) {
                    if (!fs.existsSync(dirname.slice(0, i).join('/'))) {
                        fs.mkdirSync(dirname.slice(0, i).join('/'));
                    }
                }
                fs.copyFileSync(file, file.replace(project_orig, project));
            }
            process.chdir(project);
        })
        // TODO: git checkout command
        // TODO: pull all missing remmotes
        .then(() => {
            var nodeTree = _.sortBy(gitTree(project), a => a.time);
            return displayBranches([nodeTree[0]]);
        })
        .then(r => svgs.push(r))
        //.then(() => resetAllBranches())
        .then(() => {
            process.chdir(oldDir);
            return svgs;
        })
};
module.exports = gitTipOfTree;


[Function: gitTipOfTree]

In [3]:
var importer = require('../Core');
var gitTipOfTree = importer.import('git.ipynb[tip git tree]');

$$.async();
gitTipOfTree()
    .then(r => $$.html(`<style>
div.svg-wrapper svg {
    max-width:none;
}

div.svg-output {
    min-height:600px;
}
div.svg-wrapper:first-of-type {
    position:static;
}
div.svg-wrapper {
    opacity: .05;
    position:absolute;
}

div.svg-wrapper:nth-last-of-type(3) {
    opacity: .15;
}

div.svg-wrapper:nth-last-of-type(2) {
    opacity: .3;
}

div.svg-wrapper:last-of-type {
    opacity: 1;
}
</style>
<div class="svg-output"><div
` + r.map((s, i) => ' style="left:' + ((r.length - i) * -20 + (i == r.length - 1
        ? 200
        : 0)
        + (i == r.length - 2
            ? 150
            : 0)
        + (i == r.length - 3
            ? 100
            : 0)) + 'px; top:' + (i * 10) + 'px;" class="svg-wrapper">' + s).join('</div><div') + '</div></div>'))
    .catch(e => $$.sendError(e))


reading notebook /Users/briancullinan/jupytangular2/Utilities/git.ipynb
compiling /Users/briancullinan/jupytangular2/Utilities/git.ipynb[3]
reading notebook /Users/briancullinan/jupytangular2/Frameworks/d3.ipynb
reading notebook /Users/briancullinan/jupytangular2/Utilities/git.ipynb
reading notebook /Users/briancullinan/jupytangular2/Frameworks/d3.ipynb
compiling /Users/briancullinan/jupytangular2/Frameworks/d3.ipynb[4]
compiling /Users/briancullinan/jupytangular2/Frameworks/gulp.ipynb[1]
compiling /Users/briancullinan/jupytangular2/Utilities/git.ipynb[2]
compiling /Users/briancullinan/jupytangular2/Frameworks/d3.ipynb[5]
compiling /Users/briancullinan/jupytangular2/Utilities/file system.ipynb[7]


4a626f8 c9660c1 ( swiftpage/master ) ac4a7f8 ( swiftpage/B-11243_Proof_of_Concept_for_Angular_2.0_Portal, B-11243_Proof_of_Concept_for_Angular_2.0_Portal ) fee839d a229417 6a552aa af88ea2 ( swiftpage/Only_auth, Only_auth ) a8c1fd7 2c184e7 69ea6ff ( swiftpage/B11309_APC_Direct_Purchase-Update_Preview_Api_Model ) 67b6565 7fa4cf6 6141fc7 fe71b6f ( swiftpage/B-11309_APC_Direct_Purchase-Fix_Signature_Issue ) 580ee75 bcc334f 8038f45 e3cae80 86fcb7e 386b890 ( swiftpage/B-11309_APC_Direct_Purchase, B-11309_APC_Direct_Purchase ) 4eac003 8a82bc9 498d5a5 ba08e9d 002c793 ( B11583_Portal_2_0_Sell_Pro_Direct_through_Portal_Phase_1 ) 0a5135f ( swiftpage/B-11649_Convert_Product_Ids_To_Constants, B-11649_Convert_Product_Ids_To_Constants ) 7f8ae7a ( swiftpage/Code_Model_Cleanup, Code_Model_Cleanup ) 620fcf1 ( Universal ) a116b37 ( swiftpage/B11583_Portal_2_0_Sell_Pro_Direct_through_Portal_Phase_1 ) b1619be 6fd4254 ( swiftpage/Code_Model_Cleanup_After_Pro_Support, Code_Model_Cleanup_After_Pro_Support ) 3298ca6 8ff8726 1c87962 4d9d569 d9f4daa 837e73b ( swiftpage/development ) b69b67c 0a6b2eb 94cfa6b ( HEAD -> B-11376_Account_Management_2 ) cc74fb1 f8b3dab 4ae6746 ( swiftpage/B-11650_Culture_From_URL, B-11650_Culture_From_URL )

Display graphs of different commands and scenarios using dry-run?

https://github.com/kpdecker/jsdiff

http://incaseofstairs.com/jsdiff/



In [4]:
var path = require('path');
var gulp = require('gulp');
var run = require('gulp-run');
var tap = require('gulp-tap');
var importer = require('../Core');
var gulpPromise = importer.import('gulp task to a Promise', {gulp})

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = typeof project === 'undefined' ? path.join(PROFILE_PATH, 'Documents/portal') : project;

gulp.task('git watch', function () {
    return run('git diff -w a229417..498d5a5')
        .pipe(tap(file => console.log(file)))
        .pipe(gulp.dest('output'));
});

function getBranchBoundaries(p) {
    process.chdir(project)
    project = p;
    return gulpPromise(['git watch'])
}
module.exports = getBranchBoundaries;

if(typeof $$ !== 'undefined') {
    $$.async();
    getBranchBoundaries(project)
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e));
}



compiling C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\d3.ipynb[4]
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\gulp.ipynb[1]


TypeError: gulpPromise is not a function

check github for commits since the current one?



In [2]:
var _ = require('lodash');
var importer = require('../Core');
var { gitTree, execCmd, getOauthClient, updateEvent } = importer.import([
    'd3.ipynb[format d3 tree]',
    'json git tree',
    'spawn child process',
    'import google calendar api',
    'create merge delete event'
]);

var PROFILE_PATH = process.env.HOME || process.env.HOMEPATH || process.env.USERPROFILE || '';
var project = typeof project === 'undefined' ? path.join(PROFILE_PATH, 'Documents/portal') : project;
var options = {
    calendarId: 'aws'
}

// create a calendar event to retest this branch
function installBuildTestEvent() {
    const now = new Date();
    const event = {
        summary: 'spawn child process',
        description: JSON.stringify({
            script: `npm install\nnpm run build\nnpm run test`,
            options: {cwd: project} // TODO: fix current working directory using project name
        }, null, 4),
        start: {dateTime: new Date(now.getTime())},
        end: {dateTime: new Date(now.getTime() + 30 * 60 * 1000)}
    }
    return getOauthClient(options)
        .then(() => updateEvent(event, options))            
}

function getUpdate() {
    var head = '';
    var nodeTree = _.sortBy(gitTree(project), a => a.time);
    for (const n of nodeTree) {
        if(n.branch.indexOf('HEAD') > -1) {
            // try to find a remote on the same branch
            console.log(n.branch);
            head = n.branch.split(/\s*,\s*|\s*->\s*/igm)[1].trim();
            //console.log(n);
            break;
        }
    }
    if(head.length > 0) {
        let remote;
        return execCmd(`git branch -r`, {cwd: project}) // 
            .then(r => {
                const remotes = r.join('\n').split(/\s*\n\s*/igm).filter(r => (r.split('/')[1] || '').trim() === head);
                if(remotes.length === 0) {
                    throw new Error('branch "' + head + '" not found on any remotes ' + JSON.stringify(r))
                }
                remote = remotes[0].split('/')[0];
                return remote;
            })
            .then(() => execCmd(`git fetch ` + remote, {cwd: project}))
            .then(() => execCmd('git log HEAD..' + remote + '/' + head, {cwd: project}))
            .then(log => {
                if(log.join('').trim().length > 0) {
                    return execCmd(`
git reset --hard
git pull ${remote} ${head} --rebase`, {cwd: project})
                        .then(() => installBuildTestEvent())
                } else {
                    console.log('nothing for git to do');
                }
            })
    }
    return Promise.resolve([]);
}
// git log currentbranch@{1}..currentbranch
// where the @{1} notation means "the commit the branch pointed to just before it last got updated".
// git log HEAD..swiftpage/development

if(typeof $$ !== 'undefined') {
    $$.async();
    getUpdate()
        .then(r => $$.sendResult(r))
        .catch(e => $$.sendError(e))
}


reading notebook C:\Users\brian.cullinan\Documents\jupytangular2\Utilities\git.ipynb
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Frameworks\d3.ipynb[4]
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Utilities\git.ipynb[2]
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Utilities\child process.ipynb[0]
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Utilities\google calendar.ipynb[0]
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Selenium\google authorize.ipynb[0]
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Selenium\What is Selenium.ipynb[5]
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Selenium\webdriver.ipynb[0]
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Utilities\google calendar.ipynb[13]
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Utilities\dates.ipynb[1]
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Utilities\google calendar.ipynb[7]


HEAD -> development
  swiftpage/B-11243_Proof_of_Concept_for_Angular_2.0_Portal
  swiftpage/B-11309_APC_Direct_Purchase
  swiftpage/B-11309_APC_Direct_Purchase-Fix_Signature_Issue
  swiftpage/B-11376_Account_Management_2
  swiftpage/B-11649_Convert_Product_Ids_To_Constants
  swiftpage/B-11650_Culture_From_URL
  swiftpage/B11309_APC_Direct_Purchase-Update_Preview_Api_Model
  swiftpage/Bret_Portal_Deploy_config
  swiftpage/Code_Model_Cleanup_After_Pro_Support
  swiftpage/Only_auth
  swiftpage/UI_Cleanup
  swiftpage/development
  swiftpage/master

commit 837e73bb43161890eb29544313f4c17e7df9163d
Merge: 3298ca6 4d9d569
Author: mcmahane <erika.mcmahan@swiftpage.com>
Date:   Fri Oct 27 10:57:01 2017 -0700

    Merge pull request #12 from Swiftpage/Code_Model_Cleanup
    
    Code model cleanup

commit 4d9d5695c8c0205c3491b46edd0ba363fc1e8d3f
Author: SWIFTPAGE\Brian.Cullinan <megamindbrian@gmail.com>
Date:   Fri Oct 27 10:05:50 2017 -0700

    Missing import from merge.


commit 8ff872686b3631

From https://github.com/Swiftpage/act.ecommerce
 * branch            development -> FETCH_HEAD



Updating 8a82bc9..837e73b
Fast-forward

 config/environment.staging.ts                      |   2 +-
 src/app/auth/models/form-encoded.ts                |   6 +-
 src/app/layout/language/language.component.html    |   2 +-
 src/app/layout/support/support.component.html      |   2 +-
 .../subscriptions/billing/billing.component.html   |   1 -
 src/app/subscriptions/billing/billing.component.ts |  11 +-
 .../subscriptions/billing/billing.service.spec.ts  |  28 +++--
 src/app/subscriptions/billing/billing.service.ts   |  74 ++++++-------
 src/app/subscriptions/cart/cart.component.spec.ts  |  16 ++-
 src/app/subscriptions/cart/cart.component.ts       |  38 ++++---
 .../subscriptions/confirm/confirm.component.scss   |   4 +
 .../confirm/confirm.component.spec.ts              |  25 ++---
 src/app/subscriptions/confirm/confirm.component.ts |  76 ++++++++------
 .../login-embed/login-embed.component.spec.ts      |   3 +-
 src/app/subscriptions/models/billing-model.ts      |  33 ++++++
 src/app

[ { kind: 'calendar#event',
    etag: '"3028816829358000"',
    id: '676n1rct1btiunbdn37q1ejgfo',
    status: 'confirmed',
    htmlLink: 'https://www.google.com/calendar/event?eid=Njc2bjFyY3QxYnRpdW5iZG4zN3ExZWpnZm8gNDFodGd2NnJsdm85bDc3NTluMDlqbmN0bDRAZw',
    created: '2017-12-27T21:00:14.000Z',
    updated: '2017-12-27T21:00:14.679Z',
    summary: 'spawn child process',
    description: '{\n    "script": "npm install\\nnpm run build\\nnpm run test",\n    "options": {\n        "cwd": "\\\\Users\\\\brian.cullinan\\\\Documents\\\\portal"\n    }\n}',
    creator: 
     { email: 'megamindbrian@gmail.com',
       displayName: 'Brian Cullinan' },
    organizer: 
     { email: '41htgv6rlvo9l7759n09jnctl4@group.calendar.google.com',
       displayName: 'Aws',
       self: true },
    start: { dateTime: '2017-12-27T14:00:13-07:00' },
    end: { dateTime: '2017-12-27T14:30:13-07:00' },
    iCalUID: '676n1rct1btiunbdn37q1ejgfo@google.com',
    sequence: 0,
    reminders: { useDefault: true } } ]

TODO:

Rebase squash cherrypick automatically to create perfect feature branch?

Reset each file one at a time to make sure there aren't any unnecessary changes?

Use unit testing to warn about unnecessary modules?

Load relevant projects based on branch.  It could parse the .sln file and automatically patch whenever the checked out index changes.

Plot when a TODO: item is added and when it is removed on Google Calendar, then verify by checking commit -messages, remaining comments/function names with matching keywords (split by camel casing)?, plot dates completed for measurement of self set tasks and short term goals, which activities are harder than others?  better insight in to erroneous developer time estimates?

Interface for searching all TODO: items?  Calendar?



reading notebook C:\Users\brian.cullinan\Documents\jupytangular2\Core\interpret all notebooks.ipynb
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Core\interpret all notebooks.ipynb[0]
reading notebook C:\Users\brian.cullinan\Documents\jupytangular2\Core\interpret questions.ipynb
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Core\interpret questions.ipynb[0]
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Core\interpret questions.ipynb[1]
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Core\interpret questions.ipynb[2]
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Core\interpret questions.ipynb[3]
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Core\interpret questions.ipynb[4]
compiling C:\Users\brian.cullinan\Documents\jupytangular2\Core\interpret all notebooks.ipynb[1]
reading notebook C:\Users\brian.cullinan\Documents\jupytangular2\Utilities\file system.ipynb
compiling C:\Users\brian.cullinan\Documents\jupytangular2\U

[Function: getOauthClient]